In [1]:
import os
import matplotlib.pyplot as plt
from backtests import fetch_portfolio_states, start_backtest, wait_for_backtest, BacktestRequest
from strategy import get_strategy_names

In [2]:
def plot_portfolio_value(assets_states, save_name=None, show=True):
    portfolio_values = [state.assets.equityValue for state in assets_states]
    dates = [state.createdAt for state in assets_states]
    
    plt.figure(figsize=(15, 6))
    plt.plot(dates, portfolio_values)
    plt.xlabel('Date')
    plt.ylabel('Value')
    
    if save_name is not None:
        os.makedirs("plots", exist_ok=True)
        plt.savefig(f"plots/{save_name}.png", transparent=True)
    
    if show:
        plt.show()
        
    if (save_name is not None) or show:
        plt.clf()

In [3]:
def plot_num_of_positions(assets_states):
    portfolio_values = [len(state.assets.positions) for state in assets_states]
    dates = [state.createdAt for state in assets_states]
    
    plt.figure(figsize=(15, 6))
    plt.plot(dates, portfolio_values)
    plt.xlabel('Date')
    plt.ylabel('Positions')
    plt.show()

In [4]:
class Investment:
    def __init__(self, symbol, start, end, entry, exit_value):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.entry = entry
        self.exit_value = exit_value
        
    @staticmethod
    def from_position(position, start, end):
        return Investment(position.symbol, start, end, position.averageEntryPrice * position.quantity, position.marketValue)

def get_investments(assets_states):    
    def get_last_positions(assets_states):
        result = []
        for before, after in zip(assets_states[:-1], assets_states[1:]):
            today = before.createdAt.date()
            for position in before.assets.positions:
                
                if position.symbol in [p.symbol for p in after.assets.positions]:
                    continue
                    
                result.append((position, today))
                
        for position in assets_states[-1].assets.positions:
            result.append((position, assets_states[-1].createdAt.date()))
                
        return result

    def get_open_dates(assets_states):
        result = dict()
        for before, after in zip(assets_states[:-1], assets_states[1:]):
            today = before.createdAt.date()
            for position in after.assets.positions:

                if position.symbol in [p.symbol for p in before.assets.positions]:
                    continue

                if position.symbol not in result:
                    result[position.symbol] = []

                result[position.symbol].append(today)

        return result
        
    open_dates = get_open_dates(assets_states)
    last_positions = get_last_positions(assets_states)
    
    return list(sorted([
        Investment.from_position(last, [v for v in open_dates[last.symbol] if v < end][-1], end) for last, end in last_positions
    ], key=lambda i: i.exit_value / i.entry, reverse=True))

def print_investments(investments):
    for inv in investments:
        print(f"{inv.symbol}: {inv.exit_value / inv.entry} ({inv.start} to {inv.end})")

In [5]:
class BacktestAnalysis:
    def __init__(self, total_return, investments_count, avg_investment_length, positive_investment_ratio):
        self.total_return = total_return
        self.investments_count = investments_count
        self.avg_investment_length = avg_investment_length
        self.positive_investment_ratio = positive_investment_ratio

def analyze_backtest(backtest_id, name=None, verbose=True):
    portfolio_states = fetch_portfolio_states(backtest_id)
    investments = get_investments(portfolio_states)
    test_return = portfolio_states[-1].assets.equityValue / portfolio_states[0].assets.equityValue - 1
    
    if name is None:
        name = backtest_id
        
    analysis_details = BacktestAnalysis( 
        total_return=test_return, 
        investments_count=len(investments), 
        avg_investment_length=sum((inv.end - inv.start).days for inv in investments) / len(investments) if len(investments) > 0 else 0, 
        positive_investment_ratio=len([i for i in investments if i.exit_value / i.entry > 1]) / len(investments) if len(investments) > 0 else 0
    )
    
    if verbose:
        print_analysis(name, analysis_details)
    
    plot_portfolio_value(portfolio_states, save_name=f"{name}_returns_chart", show=verbose)
    
    return analysis_details

def print_analysis(name, analysis_details):
    print(f"-> BACKTEST {name}")
    print(f"\tReturn:                     {(analysis_details.total_return*100):.3f}%")
    print(f"\tInvestments:                {analysis_details.investments_count}")
    print(f"\t Avg investment length:     {analysis_details.avg_investment_length:.3f} days")
    print(f"\t Positive investment ratio: {(analysis_details.positive_investment_ratio * 100):.2f}%")

In [6]:
def run_backtests_for_strategies(symbols, skip, prediction_error, strategies=None):
    def make_request(name):
        return BacktestRequest(
            strategy=name,
            symbols=symbols,
            skip=skip,
            use_predictor=False,
            avg_prediction_error=prediction_error
        )
    
    if strategies is None:
        strategies = get_strategy_names()
    
    for strategy_name in strategies:
        backtest_id = start_backtest(make_request(strategy_name))
        wait_for_backtest(backtest_id)
        analyze_backtest(backtest_id, name=f"err_{(prediction_error*100):.2f}_{strategy_name.lower().replace(' strategy', '').replace(' ', '_')}")

In [7]:
def run_backtests_with_folds(prediction_error):
    def make_request(name, fold):
        return BacktestRequest(
            strategy=name,
            symbols=1100,
            skip=fold*1100,
            use_predictor=False,
            avg_prediction_error=prediction_error
        )
    
    def make_name(strategy_name, fold):
        return f"5_folds_err_{(prediction_error*100):.2f}_{strategy_name.lower().replace(' strategy', '').replace(' ', '_')}_fold_{fold}"
    
    folds = list(range(5))
    for strategy_name in get_strategy_names():
        analysis_results = []
        for fold in folds:
            backtest_id = start_backtest(make_request(strategy_name, fold))
            wait_for_backtest(backtest_id)
            analysis_results.append(
                analyze_backtest(backtest_id, name=make_name(strategy_name, fold), verbose=False)
            )
            
        combined_details = BacktestAnalysis(
            total_return=sum(a.total_return for a in analysis_results) / len(analysis_results),
            investments_count=sum(a.investments_count for a in analysis_results) / len(analysis_results),
            avg_investment_length=sum(a.avg_investment_length for a in analysis_results) / len(analysis_results),
            positive_investment_ratio=sum(a.positive_investment_ratio for a in analysis_results) / len(analysis_results)
        )
        
        print_analysis(f"folds_err_{(prediction_error*100):.2f}_{strategy_name.lower().replace(' strategy', '').replace(' ', '_')}", combined_details)

### Strategy backtests 

In [ ]:
run_backtests_for_strategies(symbols=-1, skip=0, prediction_error=0)

In [ ]:
run_backtests_with_folds(prediction_error=0)

In [ ]:
for error in [0.01, 0.05, 0.1, 0.2]:
    run_backtests_with_folds(prediction_error=error)

In [ ]:
for error in [0.01, 0.05, 0.1, 0.2]:
    run_backtests_for_strategies(symbols=-1, skip=0, prediction_error=error, strategies=[
        "Basic strategy",
        "Greedy optimal strategy",
        "Overreaction strategy with predictions",
        "Trend following strategy with predictions",
        "PCA strategy with predictions"
    ])

### Bayes search

In [2]:
from bayes import perform_bayes_search
from skopt.space import Real, Integer

best_params, best_score = perform_bayes_search(
    "Basic strategy",
    {
        "maxStocksBuyCount": Integer(low=1, high=20),
        "minDaysDecreasing": Integer(low=1, high=5),
        "minDaysIncreasing": Integer(low=1, high=5),
        "topGrowingSymbolsBuyRatio": Real(0.1, 0.9)
    },
    10,
    prediction_error=0,
    symbols=200,
    skip=0
)

Running a backtest with params: {'maxStocksBuyCount': 18, 'minDaysDecreasing': 5, 'minDaysIncreasing': 5, 'topGrowingSymbolsBuyRatio': 0.776845450631919}
Waiting for backtest dd076f85-53c2-4cb0-84ef-cf5a6394cec4
Running a backtest with params: {'maxStocksBuyCount': 12, 'minDaysDecreasing': 2, 'minDaysIncreasing': 4, 'topGrowingSymbolsBuyRatio': 0.2143196729246498}
Waiting for backtest 27a6ccf6-d31a-40db-8eff-72f136e3e727
Running a backtest with params: {'maxStocksBuyCount': 10, 'minDaysDecreasing': 2, 'minDaysIncreasing': 2, 'topGrowingSymbolsBuyRatio': 0.14089230367938088}
Waiting for backtest 15baee58-7754-46b2-b49e-77834548d992
Running a backtest with params: {'maxStocksBuyCount': 9, 'minDaysDecreasing': 2, 'minDaysIncreasing': 1, 'topGrowingSymbolsBuyRatio': 0.6285123815557421}
Waiting for backtest 8b763ab9-21fe-4a18-86bf-fe4ad3697c0a
Running a backtest with params: {'maxStocksBuyCount': 5, 'minDaysDecreasing': 2, 'minDaysIncreasing': 4, 'topGrowingSymbolsBuyRatio': 0.1313775218013